In [108]:
import pandas as pd
import os
import time
import requests
import json
import csv
from tqdm import tqdm
import matplotlib.pyplot as plt

import tweepy

import requests
import pandas as pd
import os

api = pd.read_csv("../Twitter_API.txt")

In [109]:
consumer_key = api[api["Type"]=="API Key"]["Key"].to_string()[5:]
consumer_secret = api[api["Type"]=="API Key Secret"]["Key"].to_string()[5:]
access_token = api[api["Type"]=="Access Token"]["Key"].to_string()[5:]
access_token_secret = api[api["Type"]=="Access Token Secret"]["Key"].to_string()[5:]
bearer_token = api[api["Type"]=="Bearer Token"]["Key"].to_string()[5:]

In [110]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)
headers = {"Authorization": "Bearer {}".format(bearer_token)}
print()

In [111]:
import configparser
config = configparser.RawConfigParser()
config.read('../config.ini')
bearer_token = config['twitter']['bearer_token']

In [112]:
def search_twitter(query, total_number_tweets_to_get, tweet_fields, bearer_token = bearer_token):
    client = tweepy.Client(bearer_token=bearer_token)
    #query = "League of Legends"
    tweets = tweepy.Paginator(client.search_recent_tweets, query=query,
            tweet_fields=tweet_fields, max_results=100).flatten(limit=total_number_tweets_to_get)
    searched_data = []
    for tweet in tweets:
        row = [tweet.author_id, tweet.text, tweet.created_at, tweet.lang]
        searched_data.append(row)

    df_tweets = pd.DataFrame(searched_data, columns = ['author_id','text','created_at','lang'])
    return df_tweets

In [113]:
keyword_list = ['Riot','LOL','S12','World Championship','team','prediction win','win rate','KDA','gold','rank','player']

query = "League of Legends"
for val in keyword_list:
    query += val + " OR "
query = query[:-3].strip()
query

'League of LegendsRiot OR LOL OR S12 OR World Championship OR team OR prediction win OR win rate OR KDA OR gold OR rank OR player'

In [114]:
total_number_tweets_to_get = 10000
tweet_fields = ['author_id','text', 'created_at', 'lang']

## get tweets data by using defined search function and parameters
df_tweets = search_twitter(query, total_number_tweets_to_get, tweet_fields, bearer_token = bearer_token)

df_tweets

,author_id,text,created_at,lang
0,1571666106166743042,@What_If_Truth Lol,2022-09-27 02:11:56+00:00,und
1,1518454387026366465,#Paris World Challenge Cup: US shines in city ...,2022-09-27 02:11:56+00:00,en
2,342321559,@KBBasement @RexChapman I guarantee his team (...,2022-09-27 02:11:56+00:00,en
3,933356815,Check out this listing I just added to my #Pos...,2022-09-27 02:11:56+00:00,en
4,1551710898275946506,RT @CuzzyNFT: Civitas Familia is the first NFT...,2022-09-27 02:11:56+00:00,en
...,...,...,...,...
9995,1027267079639101440,@SupaStarr_ I mean….🥴 idk how this tattoo woul...,2022-09-27 02:06:34+00:00,en
9996,20875566,"Remember, you purchase the parts for the car, ...",2022-09-27 02:06:34+00:00,en
9997,1007718239298818054,RT @andrewspellman_: Big news coming from form...,2022-09-27 02:06:34+00:00,en
9998,1967936856,@jmy143 I've never seen that....heard it's goo...,2022-09-27 02:06:34+00:00,en


In [116]:
df_raw = pd.DataFrame(df_tweets)
df_raw.to_csv("../Part 2/raw_data")